In [9]:
import matplotlib.pyplot as mplt
from sklearn import svm
import numpy as np
import csv
import cv2
import os

# set matplotlib to print inline (Jupyter)
%matplotlib inline

pth = 'images/'
refactor = False

# pokemon selection

In [10]:
# Functions

def csvRead(file, filepth=pth, delimiter=','):
    with open(filepth + file, newline='') as csv_file:
        csv_data = csv.reader(csv_file, delimiter=delimiter)
        return list(csv_data)[1:]
    
def getContours(contours):
    mins_x, mins_y = list(), list()
    maxs_x, maxs_y = list(), list()
    for contour in contours:
        cnt = [c[0] for c in contour]
        cnt_x = [c[0] for c in cnt]
        cnt_y = [c[1] for c in cnt]
        mins_x.append(min(cnt_x))
        mins_y.append(min(cnt_y))
        maxs_x.append(max(cnt_x))
        maxs_y.append(max(cnt_y))

    dists_x = [ma-mi for ma, mi in zip(maxs_x, mins_x)]
    dists_y = [ma-mi for ma, mi in zip(maxs_y, mins_y)]

    areas = [x*y for x, y in zip(dists_x, dists_y)]
    index_of_largest = areas.index(max(areas))

    w = maxs_x[index_of_largest] - mins_x[index_of_largest]
    h = maxs_y[index_of_largest] - mins_y[index_of_largest]

    top_left_point = (mins_x[index_of_largest], mins_y[index_of_largest])
    bottom_right_point = (maxs_x[index_of_largest], maxs_y[index_of_largest])

    x, y = top_left_point[0], top_left_point[1]
    w, h = bottom_right_point[0] - top_left_point[0], bottom_right_point[1] - top_left_point[1]
    return (x, y, w, h)

In [11]:
if refactor:
    # Image refactoring
    # Get all pokemon directories
    pokemons = list(os.walk(pth))[0][1]

    # Convert all images to jpg
    for pokemon in pokemons:
        full_pth = pth + pokemon + '/'
        for image in os.listdir(full_pth):
            img = cv2.imread(full_pth + image, cv2.IMREAD_GRAYSCALE)
            img_color = cv2.imread(full_pth + image)
            ret, thresh = cv2.threshold(img,220,255,cv2.THRESH_BINARY_INV)
            _, contours, _ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            
            box = getContours(contours)
            max_idx = box.index(max([box[2], box[3]]))
            min_idx = box.index(min([box[2], box[3]]))
            
            img_color = img_color[box[1]: box[1] + box[max_idx], box[0]: box[0] + box[max_idx]]
            
            img_f = np.zeros((box[max_idx], box[max_idx], 3), dtype=np.uint8)
            img_f += 255
            
            left_x = box[max_idx] // 2 - img_color.shape[1] // 2
            left_y = box[max_idx] // 2 - img_color.shape[0] // 2
            
            img_f[left_y: left_y + img_color.shape[0], left_x: left_x + img_color.shape[1]] = img_color
            
            height, width = 200, 200
            img_f = cv2.resize(img_f, (width, height), interpolation = cv2.INTER_CUBIC)
            
            cv2.imwrite(full_pth + '200x200' + image, img_f)